## Use output of CTBs decision tree classifier on branchwater results
- Used branchwater with pig core species as input. 
- What kind of metaGs did branchwater return
- If we do a decision tree classifier on that, what is predictive of pig 
- How accurate is it

Manually curate the output: 
Some things are described as metagenome, gut metagenome, etc. What BioProjects are they? Are these pig/human/other?
Input is csv with [acc,prediction,org,org_count]

I also need a csv with the bioprojects: BW output probs has that. Then merge these. First go through the bioprojects on NCBI: keep a csv of [bioproject, project subject]
Should give us already a resonable amount that swithces to pig or human or whatever. 
Then for mixed bioprojects, we will need ot go through the SRA numbers. 
Keep a csv of [sra, organism_mancur]

I already have a partiallt mancur BioProject one

In [1]:
import pandas as pd
import glob 
import os
import matplotlib.pyplot as plt

In [2]:
# read the classifier predictions
df_classifier = pd.read_csv('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pig_classifier/top4_classify.nov19.csv')
len(df_classifier)

23328

In [3]:
# I need BioProject to SRA for this
def combine_branchwater_res(dfs):
    combined = pd.concat(dfs, ignore_index=True)
    combined = combined[combined['organism'] != 'NP']
    combined = combined[combined['assay_type'] == 'WGS']

    result = (
        combined
        .value_counts(['acc', 'organism', 'bioproject'])
        .reset_index(name='count')
    )

    return result

In [4]:
# set the branchwater files
files = glob.glob("/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/branchwater/*gtdb+mags.k21.csv")
# get df with counts of organism for each SRA 
# so now we have SRA to bioproject
dfs = [pd.read_csv(f, usecols=['acc', 'organism', 'assay_type', 'bioproject', 'cANI', 'containment']) for f in files]
df = combine_branchwater_res(dfs)

# Now merge with the classifier
# Keep all from the classifier df
df2 = df_classifier.merge(df[df['acc'].isin(df_classifier['acc'])], on='acc', how='left')


In [5]:
df2['acc'].nunique()

23328

# How many different metagenomes in here:
- 23328 unique metaGs. 
- 1375 Bioprojects: 20615 metaGs have a bioproject that matches any branchwater result. 
- 2713 do not have a matching branchwater bioproject. 

In [6]:
# how many diff bioprojects? 1375
df2['acc'].value_counts()
df2['bioproject'].isna().sum()


np.int64(2713)

In [7]:
# df with the manually curated bioprojects previously done 
# literally copy paste the BP number into NCBI and make notes.
df_bp = pd.read_csv('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/csv_files/251111_metag_mancur.csv')

# merge with the classifier results
df_bp = df2.merge(df_bp[df_bp['bioproject'].isin(df2['bioproject'])], on='bioproject', how='left')
df_bp = df_bp[['acc','prediction','org','org_count','bioproject','manual_org']]

# write to file if wanted
df_bp.to_csv('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pig_classifier/251119_classifier.biop.mancur.csv')

In [8]:
# about 10.000 have been previously checked. 
df_bp['manual_org'].isna().sum()



np.int64(10716)

In [10]:
# drop where mancuration was already done
df_noman = df_bp[df_bp['manual_org'].isna()]

In [13]:
df_noman['org'].value_counts().to_frame()

,count
org,
metagenome,2916
gut metagenome,2827
wastewater metagenome,1428
chicken gut metagenome,749
soil metagenome,544
...,...
Gorilla gorilla gorilla,1
Firmicutes bacterium CAG:176,1
phage metagenome,1


In [ ]:
# drop if it is an annotated something
# need to make a list bc

In [41]:
# get bioprojects bc easier man cur
df_count = (
    df_bp.groupby(['bioproject', 'org', 'manual_org'], dropna=False)
         .size()
         .reset_index(name='count')
)

# using this for first mancur step
df_count.to_csv('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pig_classifier/251119_bioproject.counts.csv')

In [42]:
len(df_count)

1488